In [ ]:
import sqlite3 as sql
import sql_utils as u
from pathlib import Path
import pandas as pd

%load_ext autoreload
%autoreload 2

# Pokemon DB

## Initialize from SQL files

In [ ]:
POKEMON_DB = Path(r"resources\pokemon.db")

def query(q: str, connection: sql.Connection=None) -> pd.DataFrame:
    q = q.strip()
    if connection is None:
        connection = sql.connect(POKEMON_DB)
    results = pd.read_sql_query(q, connection)
    connection.close()
    return results

# write tables
u.initialize_database(POKEMON_DB, [Path("resources/sql_scripts/pokemon_db/create_tables.sql")])

# insertion sql statements
sql_paths = list(Path("resources/sql_scripts/pokemon_db").glob("insert_*.sql"))

u.initialize_database(POKEMON_DB, sql_paths)

In [ ]:
# Some cleanup to align names between sources
with sql.connect(POKEMON_DB) as conn:
    cur = conn.cursor()
    cur.execute(f"UPDATE trainers SET pokemon = LOWER(pokemon);")
    cur.execute(f"UPDATE pokemon SET pok_name = 'basculin' WHERE pok_name = 'basculin-red-striped';")
    cur.execute(f"UPDATE trainers SET pokemon = 'nidoran-m' WHERE pokemon = 'nidoran♂';")
    conn.commit()

In [ ]:
u.list_table_schemas(POKEMON_DB)

## Queries

In [ ]:
# Confirm that the trainers_pokemon are all in the pokedex
# note that this wasn't the case before and I fixed that in the sql files

q = """
SELECT t.*
FROM trainers as t
where t.pokemon NOT IN (
    SELECT pok_name from pokemon
)
"""

query(q)

In [ ]:
# pokemon ids for trainers pokemon

query("""
SELECT t.*, pk.pok_id
FROM trainers as t
JOIN pokemon as pk on pk.pok_name = t.pokemon
LIMIT 10
""")

In [ ]:
# try to figure out how the pokedex stats relate to trainers pokemon stats

query("""
SELECT t.*, b.*
FROM trainers as t
JOIN pokemon as pk on pk.pok_name = t.pokemon
JOIN base_stats as b on b.pok_id = pk.pok_id
ORDER BY b.b_atk + b.b_def DESC
LIMIT 20
""")

# Natures and IVs and EVs make this hard to figure out